# Introduction to programming quantum computers using pyQuil
## Warsaw Quantum Computing meeting

## Installations

### Virtualenv

First, create a new virtualenv:

`$virtualenv -p python3 warsaw-qc-venv`,

`$source warsaw-qc-venv/bin/activate`.

Then allow jupyter to use this venv:

`$pip install ipykernel`

and

`$ipython kernel install --user --name=warsaw_qc`

(source: https://anbasile.github.io/programming/2017/06/25/jupyter-venv/)

### Getting your environment ready

Ok, so we need to follow the ["Getting started" guide](https://pyquil.readthedocs.io/en/stable/start.html) and do the following:

- install pyQuil (`pip install pyquil`)
- download QVM and compiler - follow the instructions from [the guide](https://pyquil.readthedocs.io/en/stable/start.html)
- run 'qvm -S' in one console
- run 'quilc -S' in second console

Additionally, in this tutorial we also need scipy and qutip:
- `pip install scipy`
- `pip install qutip`

### Visualization tools

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

def plot_state_histogram(result):
    total_count = len(result[0])
    n_qubits = len(result)
    all_states = []
    
    for i in range(total_count):
        state = []
        for j in range(n_qubits):
            state.append(result[j][i])
        all_states.append(tuple(state))

    states_with_counts = Counter(all_states).most_common()[::-1]    
    
    states = np.array(states_with_counts)[:,0]
    probs = (np.array(states_with_counts)[:,1]/total_count).astype(float)
    n = len(states_with_counts)
    plt.barh(range(n), probs, tick_label=states)
    plt.show()


In [ ]:
import cmath
from qutip import Bloch
from pyquil.api import WavefunctionSimulator


def get_vector(alpha, beta):
    """
    Function to compute 3D Cartesian coordinates
    from 2D qubit vector.
    """

    # get phases
    angle_alpha = cmath.phase(alpha)
    angle_beta = cmath.phase(beta)

    # avoiding wrong normalization due to rounding errors
    if cmath.isclose(angle_alpha, cmath.pi):
        angle_alpha = 0
    if cmath.isclose(angle_beta, cmath.pi):
        angle_beta = 0
        
    if (angle_beta < 0 and angle_alpha < angle_beta) or (angle_beta > 0 and angle_alpha > angle_beta):
            denominator = cmath.exp(1j*angle_beta)
    else:
            denominator = cmath.exp(1j*angle_alpha)

    # eliminate global phase
    alpha_new = alpha/denominator
    beta_new = beta/denominator

    # special case to avoid division by zero
    if abs(alpha) == 0 or abs(beta) == 0:
        if alpha == 0:
            return [0,0,-1]
        else:
            return [0,0,1]
    else:
        # compute theta and phi from alpha and beta
        theta = 2*cmath.acos(alpha_new)
        phi = -1j*cmath.log(beta_new/cmath.sin(theta/2))

        # compute the Cartesian coordinates
        x = cmath.sin(theta)*cmath.cos(phi)
        y = cmath.sin(theta)*cmath.sin(phi)
        z = cmath.cos(theta)

    return [x.real,y.real,z.real]

def plot_quantum_state(program):
    """
    Thin function to abstract the plotting on the Bloch sphere.
    """
    wfn = WavefunctionSimulator().wavefunction(program)
    amplitudes = wfn.amplitudes
    print(amplitudes)
    bloch_sphere = Bloch()
    vec = get_vector(amplitudes[0], amplitudes[1])
    bloch_sphere.add_vectors(vec)
    bloch_sphere.show()
    bloch_sphere.clear()


## Basic programs 

In [ ]:
from pyquil import Program, get_qc
from pyquil.gates import *
from pyquil.api import WavefunctionSimulator

In [ ]:
# construct an empty program
p = Program()
qc = get_qc('2q-qvm')
result = qc.run_and_measure(p, trials=10)
print(result)

In [ ]:
def run_and_visualize(program, n_qubits=2, trials=10, qc=None):
    if qc is None:
        qc = get_qc(str(n_qubits)+'q-qvm')
    
    result = qc.run_and_measure(p, trials=trials)
    if trials <= 100:
        for i in range(n_qubits):
            print(result[i])
    wfn = WavefunctionSimulator().wavefunction(p)
    print("wavefunction:", wfn)
    print("Probabilities:", wfn.probabilities())
    plot_state_histogram(result)

In [ ]:
# construct an equal superposition
p = Program(H(0))
run_and_visualize(p, n_qubits=1, trials=100)

In [ ]:
# construct an equal superposition on N qubits
p = Program()
N = 5
for i in range(N):
    p += (H(i))
print(p)
run_and_visualize(p, n_qubits=N, trials=100)

In [ ]:
# construct a Bell State (|00> + |11>)
p = Program(H(0), CNOT(0, 1))

In [ ]:
# construct a GHZ state (|00...0> + |11...1>)
N = 5
p = Program(H(0))
for i in range(1, N):
    p += Program(CNOT(0, i))
run_and_visualize(p, n_qubits=N)

### Exploring gates

In [ ]:
# Let's explore some basic gates: I, X, Y, Z, RX, RY, RZ, H, CNOT, SWAP

In [ ]:
p = Program(H(0))
run_and_visualize(p,1)

In [ ]:
plot_quantum_state(p)

## A little bit more complex example

In [ ]:
p = Program(H(0))
# p = Program(H(0), RZ(np.pi/2, 0))
# p += CNOT(0, 1)
# p += H(0)
# p += H(1)
run_and_visualize(p, 2)

## Measurements

In [ ]:
qc = get_qc('2q-qvm')
p = Program()
p += H(0)
result = qc.run_and_measure(p, trials=10)
print(result[0])

In [ ]:
qc = get_qc('2q-qvm')
p = Program()
p += H(0)
#ro is short for readout
ro = p.declare('ro', 'BIT', 16)
p += MEASURE(0, ro[0])
# p += MEASURE(1, ro[1])
result = qc.run(p)

print(result)

In [ ]:
qc = get_qc('2q-qvm')
p = Program()
p += X(0)
#ro is short for readout
ro = p.declare('ro', 'BIT', 16)
p += MEASURE(0, ro[0])
p += H(0)
result = []
for i in range(100):
    result.append(qc.run(p)[0,0])

print(np.array(result))

In [ ]:
qc = get_qc('2q-qvm')
p = Program()
p += X(0)
p += H(1)
#ro is short for readout
ro = p.declare('ro', 'BIT', 16)
p += MEASURE(0, ro[0])
# p += MEASURE(1, ro[1])
# p += H(0)
p.wrap_in_numshots_loop(100)
result = qc.run(p)

print(result[:])

## Noise

In [ ]:
qc = get_qc('2q-qvm', noisy=True)
p = Program(H(0))
run_and_visualize(p, n_qubits=1, qc=qc)

In [ ]:
qc = get_qc('2q-qvm', noisy=True)
p = Program(X(0))
run_and_visualize(p, n_qubits=1, qc=qc, trials=100)

In [ ]:
p = Program(X(0))
qc = get_qc('2q', as_qvm=True, noisy=True)
result = qc.run_and_measure(p, trials=10000)
print(result[0].sum())

In [ ]:
run_and_visualize(p, n_qubits=1, qc=qc, trials=100)

In [ ]:
qc = get_qc('3q', as_qvm=True, noisy=True)
p = Program(H(0), H(1), H(2))
run_and_visualize(p, n_qubits=3, qc=qc,trials=10000)

In [ ]:
qc = get_qc('3q', as_qvm=True, noisy=False)
p = Program(H(0), H(1), H(2))
run_and_visualize(p, n_qubits=3, qc=qc,trials=10000)

### Read the docs: [More about noise models](https://pyquil.readthedocs.io/en/stable/apidocs/noise.html)

## Connectivity

In [ ]:
from pyquil import list_quantum_computers
list_quantum_computers()

In [ ]:
qc = get_qc('Aspen-1-2Q-B')
# qc = get_qc('Aspen-1-2Q-B',as_qvm=True)
# qc = get_qc('Aspen-1-9Q-B',as_qvm=True)
# qc = get_qc('Aspen-1-15Q-A',as_qvm=True)
p = Program(X(0))
run_and_visualize(p, n_qubits=1, qc=qc, trials=100)

In [ ]:
import networkx as nx
def visualize_topology(qc_name):
    qc = get_qc(qc_name, as_qvm=True)
    print("nodes:", qc.qubit_topology().nodes)
    print("edges:", qc.qubit_topology().edges)
    nx.draw(qc.qubit_topology())
    from matplotlib import pyplot as plt
    _ = plt.title(qc_name, fontsize=18)

In [ ]:
visualize_topology('Aspen-1-2Q-B')

In [ ]:
# qc = get_qc('17q', as_qvm=True)
qc = get_qc('Aspen-1-5Q-B', as_qvm=True)
p = Program()
p += H(0)
p += CNOT(0, 14)
p += CNOT(0, 1)
# p += CNOT(0, 16)
# p += CNOT(0, 15)
print(qc.compile(p).program)

### Read the docs: [Hierararchy of realism](http://docs.rigetti.com/en/stable/migration2-qc.html#Heirarchy-of-realism)

- `WavefunctionSimulator` to debug algorithm
- `get_qc("5q-qvm")` to debug sampling
- `get_qc("9q-square-qvm")` to debug mapping to a lattice
- `get_qc("9q-square-noisy-qvm")` to debug generic noise characteristics
- `get_qc("Aspen-0-16Q-A-qvm")` to debug mapping to a real lattice
- `get_qc("Aspen-0-16Q-A-noisy-qvm")` to debug noise characteristics of a real device
- `get_qc("Aspen-0-16Q-A")` to run on a real device



### Read the docs: [Providing your own chip topology](http://docs.rigetti.com/en/stable/qvm.html#providing-your-own-device-topology)

## Parametric compilation

In [ ]:
p = Program()
ro = p.declare("ro", "BIT", 1)
theta_ref = p.declare("theta", "REAL")

p += RX(np.pi / 2, 0)
p += RZ(theta_ref, 0)
p += RX(-np.pi / 2, 0)
p += MEASURE(qubit, ro[0])


In [ ]:
# Get a Quantum Virtual Machine to simulate execution
qc = get_qc("1q-qvm")
executable = qc.compile(p)
print(executable.program)


In [ ]:
parametric_measurements = []

for theta in np.linspace(0, 2 * np.pi, 200):
    # Get the results of the run with the value we want to execute with
    bitstrings = qc.run(executable, {'theta': [theta]})
    # Store our results
    parametric_measurements.append(bitstrings)


In [ ]:
p = Program()
p += RX(np.pi / 2, 0)
p += RZ(0, qubit)
p += RX(-np.pi / 2, 0)

print(p)
plot_quantum_state(p)

### Read the docs: [Parametric compilation](http://docs.rigetti.com/en/stable/basics.html#parametric-compilation)

## Simple optimization example

[Gradient descent source](https://towardsdatascience.com/implement-gradient-descent-in-python-9b93ed7108d1)

In [ ]:
p = Program()
ro = p.declare("ro", "BIT", 3)
theta_0_ref = p.declare("theta_0", "REAL")
p += RX(theta_0_ref, 0)
p += MEASURE(0, ro[0])
p.wrap_in_numshots_loop(1000);

In [ ]:
qc = get_qc("1q", as_qvm=True, noisy=False)
executable = qc.compile(p)
print(executable.program)

In [ ]:
bitstrings = qc.run(executable, {'theta_0': [0]})
bitstrings.sum()/1000

In [ ]:
theta_0 = np.pi # The algorithm starts at pi
rate = 0.01 # Learning rate
precision = 1e-5 #This tells us when to stop the algorithm
previous_step_size = 1 #
max_iters = 1000 # maximum number of iterations
iters = 0 #iteration counter
def cost_fun(x):
    return 2*(x - 0.5)


In [ ]:
while previous_step_size > precision and iters < max_iters:
    prev_theta_0 = theta_0 #Store current x value in prev_x
    bitstrings = qc.run(executable, {'theta_0': [theta_0]})
    y = bitstrings.sum() / 100
    theta_0 = theta_0 - rate * cost_fun(y) #Grad descent
    previous_step_size = abs(theta_0 - prev_theta_0) #Change in x
    iters = iters + 1 #iteration count
    if iters%20 == 0:
        print("Iteration",iters,"\nY:",y, "\nTheta:", theta_0 / np.pi, "pi") #Print iterations
    
print("The optimal value of theta:", np.round(theta_0/np.pi,3), "pi")
print("Number of iterations:", iters)


## Further materials

- My tutorial on solving Travelling Salesman Problem: https://github.com/mstechly/quantum_tsp_tutorials
- I re-used parts of [this tutorial](https://github.com/markf94/rigetti_training_material/blob/master/session_2/tutorial_II_quantum_gates.ipynb) by Mark Fingerhuth and Tomas Babej from [ProteinQure](https://proteinqure.com)
- [Quantum Open Source Foundation](https://www.qosf.org) - many good learning materials here
- [Someone shouts |01000>. Who get's excited?](https://arxiv.org/pdf/1711.02086.pdf) - a short article about the notation used in pyQuil
- [bqResearcher program](https://www.bohr.technology/bqresearcher)
- [YT playlist with some interesting videos about QC](https://www.youtube.com/playlist?list=PLpQk8lG_JZSrgMdQK6Tibmk8EpISYak3P)